# Animations with Earth Engine
## Purpose
The purpose of this notebook is to animations from Earth Engine datasets.

## Setup
### Library import

In [1]:
import os
from ipywidgets import interact
import matplotlib.pyplot as plt

from vizzDL import animations
from vizzDL.models.CNN.super_resolution import srgan

## **Create animation**
**Create `animations` object**

In [2]:
animation = animations.Animation()

**Select region**

In [3]:
animation.select_region(instrument='Landsat', lat=-14.66073, lon=17.69604, zoom=12)

Draw a rectangle on map to select and area.


Map(center=[-14.66073, 17.69604], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

### Time-lapse
**Create animation as a Numpy array**

In [4]:
video = animation.video_as_array(start_year=1988, stop_year=1990)

Image number: 0
Image number: 1
Image number: 2


### Prediction
**Create model**

In [5]:
# Location of model weights
weights_dir = '../../datasets/processed/Models/L8_S2_SR_x3/srgan_generator_L8_to_S2_x3'
weights_file = lambda filename: os.path.join(weights_dir, filename)

pre_generator = srgan.Generator(input_shape=(None, None, 3), scale=3).generator()
pre_generator.load_weights(weights_file('model_weights.h5'))

**predict**

In [6]:
prediction = animation.predict(model = pre_generator, norm_range=[[0,1], [-1,1]])

In [7]:
@interact(frame=(0, video.shape[0]-1))
def show_frame(frame=0):
    fig, ax = plt.subplots(1, 2, figsize=(20,20))

    ax[0].imshow(video[frame,:,:,:])
    ax[0].set_title('LR')

    ax[1].imshow(prediction[frame,:,:,:])
    ax[1].set_title('SR (PRE)')

interactive(children=(IntSlider(value=0, description='frame', max=2), Output()), _dom_classes=('widget-interac…

### Create tiles
**Create tiles**

In [8]:
animation.create_tiles(folder_path = '../../datasets/processed/Tiles/', region_name = 'Menongue')